In [1]:
import sys
import os

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))
sys.path.insert(0, SOURCE_DIR)

In [2]:
import malaya_speech
import numpy as np
from malaya_speech import Pipeline
from malaya_speech.utils.astype import float_to_int

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [11]:
from glob import glob
from unidecode import unidecode
import os
import torchaudio
import torch

youtubes = sorted(glob('/home/husein/ssd2/noise/*.mp3'))
youtubes.extend(glob('/home/husein/ssd2/noise/Nonspeech/*.mp3'))
youtubes = [f for f in youtubes if 'NCS Release' not in f]
len(youtubes)

529

In [12]:
parent_dir = '/home/husein/ssd2/processed-noise'
# !rm -rf {parent_dir}
!mkdir {parent_dir}

parent_dir_done = '/home/husein/ssd2/processed-noise-done'
# !rm -rf {parent_dir_done}
!mkdir {parent_dir_done}

mkdir: cannot create directory ‘/home/husein/ssd2/processed-noise’: File exists
mkdir: cannot create directory ‘/home/husein/ssd2/processed-noise-done’: File exists


In [13]:
from tqdm import tqdm

for f in tqdm(youtubes):

    new_f = unidecode(os.path.split(f)[1].replace('.mp3', '').replace(' ', '_'))
    f_done = os.path.join(parent_dir_done, new_f)

    if os.path.exists(f_done):
        continue
        
    samples = malaya_speech.streaming.torchaudio.stream(f,
                                                    segment_length = 882,
                                                    realtime_print = False,
                                                    sample_rate = 44100,
                                                    min_length = 2.0,
                                                    max_length = 16.0,
                                                   )
    if len(samples):
        parent_new_f = os.path.join(parent_dir, new_f)
        os.makedirs(parent_new_f, exist_ok=True)

        for i in range(len(samples)):
            audio_path = os.path.join(parent_new_f, f'{i}.mp3')
            torchaudio.save(audio_path, 
                            torch.tensor(samples[i]['wav_data'].astype('float32')).unsqueeze(0), 
                            44100, format='mp3')

    with open(f_done, 'w') as fopen:
        fopen.write('done')

100%|█████████████████████████████████████████| 529/529 [38:18<00:00,  4.35s/it]


In [15]:
files = glob('/home/husein/ssd2/processed-noise/*/*.mp3')

len(files)

29801

In [16]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [17]:
from tqdm import tqdm

ls = 0
for x in tqdm(files):
    ls += (len(audio.decode_example(audio.encode_example(x))['array']) / sr)

100%|█████████████████████████████████████| 29801/29801 [07:45<00:00, 64.08it/s]
